<div align="center">
    
# Как поступать с отсутсвующими данными?

</div>

---

Мы рассмотрим несколько практических методов решения проблемы
пропущенных значений путем удаления неполных записей из набора данных или подстановки пропущенных значений из других обучающих экземпляров и признаков.



In [34]:
import pandas as pd
from io import StringIO

# Создаём строку, имитирующую CSV-файл
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

# Читаем CSV-данные с помощью StringIO
df = pd.read_csv(StringIO(csv_data))

# Выводим DataFrame
print(df)

      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN


Мы использовали `pd.read_csv` вместе с `StringIO`, чтобы загрузить CSV-данные из строки в `DataFrame`. Отсутствующие значения автоматически заменились на `NaN`. `StringIO` здесь просто имитирует файл, позволяя работать со строкой как с CSV-файлом.

В больших DataFrame искать пропущенные значения вручную неудобно. Вместо этого используем метод `isnull()`, который показывает, где значения отсутствуют (`True`). А с помощью `sum()` можно посчитать количество таких пропусков в каждом столбце.

In [35]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

Для удобной предобработки можно использовать DataFrame из pandas, но scikit-learn всё ещё лучше работает с массивами NumPy. Поэтому перед передачей данных в модель лучше использовать .values, чтобы получить NumPy-массив из DataFrame:

In [36]:
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

## Исключение обучающих записей или признаков с пропущенными значениями

Один из простых способов обработки пропущенных данных — их удаление: строк (`axis=0`) или столбцов (`axis=1`). Метод `dropna()` позволяет это сделать, а также имеет гибкие параметры:

* `how='all'` — удаляет строки, где **все** значения пропущены.
* `thresh=n` — удаляет строки, в которых менее **n** реальных значений.
* `subset=[...]` — удаляет строки, если **NaN есть в указанных столбцах**.

In [37]:
# Удаляет все  строки с пропущенными значениями
df.dropna(axis = 0)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [38]:
# Удаляет все столбцы с пропущенными значениями
df.dropna(axis = 1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [39]:
# Возвращает массив, в котором нет строк, содержащих только значения NaN
df.dropna(how = 'all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [40]:
# Мы указали, что должны вернуться строки с 4 значениями
df.dropna(thresh = 4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [41]:
# Задали столбец C, удалит строки в которых содержится NaN
df.dropna(subset = ('C'))

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


Удаление пропущенных данных удобно, но опасно: можно потерять слишком много записей или важных признаков, что ухудшит качество анализа и работы модели, поэтому лучше рассмотреть метод интерполяции для работы с пропущенными значениями.

## Подстановка пропущенных значений

Одним из наиболее распространенных методов интерполяции является подстановка среднего (mean imputation), когда мы просто заменяем отсутствующее значение средним значением всего столбца признаков. Удобный способ сделать это - использовать класс Simpleimputer из scikit-learn.

In [42]:
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

Мы заменили NaN средним по столбцу. Можно также использовать медиану **(median)** или самое частое значение **(most_frequent)** — особенно удобно для категориальных признаков, например, с цветами.

Альтернативным и еще более удобным способом подстановки отсутствующих значений является применение метода **fillna** библиотеки pandas и передача метода подстановки в качестве аргумента.

In [43]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


## API оцениватели scikit-learn

`SimpleImputer` из scikit-learn — это инструмент для автоматической подстановки пропущенных значений. Он входит в так называемый **transformer API**, который используется для обработки и преобразования данных в машинном обучении.

У этого класса два основных метода:

* **`fit()`** — анализирует обучающие данные и вычисляет параметры для подстановки (например, среднее или медиану по столбцам).
* **`transform()`** — применяет эти параметры для замены пропущенных значений в новых данных.

Важно, чтобы при применении `transform()` структура (количество признаков) совпадала с той, что использовалась в `fit()`. Это позволяет точно и последовательно заполнять пропуски в тренировочных и тестовых данных.

Таким образом, `SimpleImputer` помогает стандартизировать обработку пропущенных значений в соответствии с выбранной стратегией, облегчая подготовку данных для обучения моделей.
